# Experiment 2

#### Model Setup

Run the Multilabel Stereotype + Omission Document Classifier without any additional labels from other classifiers as features.

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[I.](#i) Stereotype + Omission Classifier
* [Preprocessing](#prep)
* [Training & Prediction](#tp)
* [Evaluation](#eval)

[II.](#ii) Predict Over All Data

Load programming resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For classification
import scipy
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix#, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support

# For saving model
from joblib import dump,load

Define resources for the models:

In [2]:
### For 60-20-20 train-dev-test split of documents (metadata descriptions)
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
# Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
# Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics
# ----------------------
### For features from modified 5-fold CV
# predictions_dir = config.experiment2_path+"5fold/output/"
# Path(predictions_dir).mkdir(parents=True, exist_ok=True)    # For predictions
# agreement_dir = config.experiment2_path+"5fold/agreement/"
# Path(agreement_dir).mkdir(parents=True, exist_ok=True)      # For agreement metrics

# For baseline
predictions_dir = config.tokc_path+"baseline/output/"
Path(predictions_dir).mkdir(parents=True, exist_ok=True)    # For predictions
agreement_dir = config.tokc_path+"baseline/agreement/"
Path(agreement_dir).mkdir(parents=True, exist_ok=True)      # For agreement metrics

# predictions_dir = config.experiment2_path+"5fold/with_manual_labels/output/"              # For predictions with features as manual labels
# Path(predictions_dir).mkdir(parents=True, exist_ok=True)  
# agreement_dir = config.experiment2_path+"5fold/with_manual_labels/agreement/"             # For agreement metrics with features as manual labels
# Path(agreement_dir).mkdir(parents=True, exist_ok=True)

In [3]:
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100               # dimensions of word embeddings (should match utils1.py) for file names
target_labels = "so"  # for file names

<a id="i"></a>
## I. Stereotype + Omission Classifier
<a id="prep"></a>
### Preprocessing

Load the document classification model's input data:

In [7]:
# # For 60-20-20 split
# train = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_train.csv".format(target_labels), index_col=0)
# dev = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_validate.csv".format(target_labels), index_col=0)
# test = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_test.csv".format(target_labels), index_col=0)
# df_exp = pd.concat([train, dev, test])
# df_exp["label"] = df_exp["label"].fillna("{'None'}")
# df_exp = df_exp.loc[~df_exp.description.isna()]
# df_exp = utils.getColumnValuesAsLists(df_exp, "label")
# # df_exp.head()
# ------------------------------
# For modified 5-fold cross validation
df = pd.read_csv(config.tokc_path+"experiment_input/document_5fold.csv", index_col=0)
df = utils.getColumnValuesAsLists(df, "label")
df = df.drop(columns=["subset"])
df.head()

,description_id,start_offset,end_offset,field,description,label,fold
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3


Define the train (80% of the data) and test (20% of the data) splits:

In [8]:
split_col = "fold"
splits = df[split_col].unique()
splits.sort()
print(splits)
train0, test0 = list(splits[:4]), splits[4]
train1, test1 = list(splits[1:]), splits[0]
train2, test2 = list(splits[2:])+[splits[0]], splits[1]
train3, test3 = list(splits[3:])+list(splits[:2]), splits[2]
train4, test4 = [splits[4]]+list(splits[:3]), splits[3]
runs = [(train0, test0), (train1, test1), (train2, test2), (train3, test3), (train4, test4)]
for run in runs:
    print(run)

['split0' 'split1' 'split2' 'split3' 'split4']
(['split0', 'split1', 'split2', 'split3'], 'split4')
(['split1', 'split2', 'split3', 'split4'], 'split0')
(['split2', 'split3', 'split4', 'split0'], 'split1')
(['split3', 'split4', 'split0', 'split1'], 'split2')
(['split4', 'split0', 'split1', 'split2'], 'split3')


In [9]:
def binarizeMultilabelTrainColumn(df_col):
    mlb = MultiLabelBinarizer()
    binarized = mlb.fit_transform(df_col)
    return mlb, binarized

def binarizeMultilabelDevColumn(mlb, df_col):
    binarized = mlb.transform(df_col)
    return binarized

<a id="tp"></a>
### Train & Predict

In [10]:
a = "sgd-svm"

In [12]:
pred_df = pd.DataFrame()
target_col = "label"
for run in runs:
    # Get the train (80%) and test (20%) subsets of data
    train_splits, test_split = run[0], run[1]
    print("Training on:", train_splits)
    train_df = df.loc[df[split_col].isin(train_splits)]
    dev_df = df.loc[df[split_col] == test_split]
    
    # Vectorize the documents (descriptions)
    cvectorizer = CountVectorizer()
    tfidf = TfidfTransformer()
    train_docs = cvectorizer.fit_transform(train_df["description"])
    dev_docs = cvectorizer.transform(dev_df["description"])
    X_train = tfidf.fit_transform(train_docs)
    X_dev = tfidf.transform(dev_docs)
    
    # Binarize targets
    mlb_target, y_train = binarizeMultilabelTrainColumn(train_df[target_col])
    y_dev = binarizeMultilabelDevColumn(mlb_target, dev_df[target_col])

    # Train a classification model
    clf = OneVsRestClassifier(SGDClassifier(loss="hinge"))  # Support Vector Machines loss function
    clf.fit(X_train, y_train)
    
    # Predict with the trained model
    print("Predicting on:", test_split)
    predictions = clf.predict(X_dev)
    pred_labels = mlb_target.inverse_transform(predictions)    
    if pred_df.shape[0] > 0:
        next_pred_df = dev_df.copy()
        next_pred_df.insert(len(next_pred_df.columns), "{}_label".format(a), pred_labels)
        pred_df = pd.concat([pred_df, next_pred_df])
    else:
        pred_df = dev_df.copy()
        pred_df.insert(len(pred_df.columns), "{}_label".format(a), pred_labels)

print("Modified 5-fold cross-validation complete!")
print(pred_df.shape)

Training on: ['split0', 'split1', 'split2', 'split3']
Predicting on: split4
Training on: ['split1', 'split2', 'split3', 'split4']
Predicting on: split0
Training on: ['split2', 'split3', 'split4', 'split0']
Predicting on: split1
Training on: ['split3', 'split4', 'split0', 'split1']
Predicting on: split2
Training on: ['split4', 'split0', 'split1', 'split2']
Predicting on: split3
Modified 5-fold cross-validation complete!
(27312, 8)


In [13]:
pred_df = pred_df.rename(columns={"label":"manual_label"})
pred_df.head()

,description_id,start_offset,end_offset,field,description,manual_label,fold,sgd-svm_label
6,3027,627,1162,Biographical / Historical,Thomas Young was probably born in 1725. By the...,[Stereotype],split4,"(Stereotype,)"
7,3397,8095,8334,Biographical / Historical,Andrew Tait worked on Paramecium in Beale's la...,[Omission],split4,"(Omission,)"
10,4736,9951,10026,Biographical / Historical,Delivered by Thomson to teachers in Darlington.,[Omission],split4,"(Omission,)"
14,4712,4199,4485,Biographical / Historical,"This was gifted by Thomson to his secretary, M...",[Omission],split4,"(Omission,)"
22,15684,845,1179,Biographical / Historical,Catherine Robina Borland was responsible for t...,[],split4,"(,)"


Save the predictions data:

In [14]:
pred_df.to_csv(predictions_dir+"aggregated_final_validate_predictions_docclf_{a}_{t}.csv".format(a=a, t=target_labels))

Save the model:

In [15]:
model_dir = "models/baseline/"
Path(model_dir).mkdir(parents=True, exist_ok=True)
filename = model_dir+"sgd-svm_F-tfidf_T-so.joblib"
dump(clf, filename)

['models/baseline/sgd-svm_F-tfidf_T-so.joblib']

<a id="eval"></a>
### Evaluate

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [16]:
classes = clf.classes_  #doc_clf.classes_
print(classes)
original_classes = mlb_target.classes_
print(original_classes)
label_dict = dict(zip(original_classes, classes))

[0 1 2]
['' 'Omission' 'Stereotype']


Create a [confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-confusion-matrix) of the results, where, for class *i*:
* Count of true negatives (TN) is at position *i*,0,0
* Count of false negatives (FN) is at position *i*,1,0
* Count of true positives (FP) is at position *i*,1,1
* Count of false positives (PF) is at position *i*,0,1

In [17]:
y_dev = binarizeMultilabelDevColumn(mlb_target, pred_df["manual_label"])
predictions = mlb_target.transform(pred_df["{}_label".format(a)])
assert len(y_dev) == len(predictions)

In [18]:
matrix = multilabel_confusion_matrix(y_dev, predictions, labels=classes)

In [19]:
scores = utils.getPerformanceMetrics(y_dev, predictions, matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,1857,2175,356,0.859344,0.539435,0.662807
2,Stereotype,406,1195,77,0.939465,0.746408,0.831883


Save the performance results:

In [20]:
scores.to_csv(agreement_dir+"docclf_{a}_{t}_baseline_performance.csv".format(a=a, t=target_labels))